In [95]:
import sys
path = sys.executable

In [ ]:
!/usr/local/bin/python3 -m pip install selenium-stealth
!/usr/local/bin/python3 -m  pip install fake-useragent

In [136]:
from selenium import webdriver
from selenium_stealth import stealth
import time
from selenium.webdriver.chrome.service import Service
from fake_useragent import UserAgent
from bs4 import BeautifulSoup

'''
https://stackoverflow.com/questions/32391303/how-to-scroll-to-the-end-of-the-page-using-selenium-in-python
'''
def scroll_to_bottom(driver):

    old_position = 0
    new_position = None

    while new_position != old_position:
        # Get old scroll position
        old_position = driver.execute_script(
                ("return (window.pageYOffset !== undefined) ?"
                 " window.pageYOffset : (document.documentElement ||"
                 " document.body.parentNode || document.body);"))
        # Sleep and Scroll
        time.sleep(3)
        driver.execute_script((
                "var scrollingElement = (document.scrollingElement ||"
                " document.body);scrollingElement.scrollTop ="
                " scrollingElement.scrollHeight;"))
        # Get new position
        new_position = driver.execute_script(
                ("return (window.pageYOffset !== undefined) ?"
                 " window.pageYOffset : (document.documentElement ||"
                 " document.body.parentNode || document.body);"))

def scroll_slow(driver):

    lenOfPage = driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
    match=False
    while(match==False):
            lastCount = lenOfPage
            time.sleep(3)
            lenOfPage = driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
            if lastCount==lenOfPage:
                match=True


In [140]:
import urllib.parse 
def is_relative_url(url):
        parsed_url = urllib.parse.urlparse(url)
        return not parsed_url.scheme and not parsed_url.netloc

In [148]:

def get_html(base_url):
        
        ua = UserAgent()
        user_agent = ua.random

        options = webdriver.ChromeOptions()
        options.add_argument("start-maximized")
        options.add_argument(f'--user-agent={user_agent}')
        options.add_experimental_option("excludeSwitches", ["enable-automation"])
        options.add_experimental_option('useAutomationExtension', False)
        
        service = Service(executable_path='/Users/adautov/Downloads/chromedriver-mac-arm64/chromedriver')

        driver = webdriver.Chrome(options=options, service=service)

        stealth(driver,
                languages=["en-US", "en"],
                vendor="Google Inc.",
                platform="Win32",
                webgl_vendor="Intel Inc.",
                renderer="Intel Iris OpenGL Engine",
                fix_hairline=True,
                )

        driver.get(base_url)
        driver.implicitly_wait(10)  # wait for 10 seconds
        
        scroll_to_bottom(driver)
        
        page_source = driver.page_source
        driver.quit()
        return page_source


In [ ]:
base_url = "https://www.ozon.ru/category/smartfony-15502/?category_was_predicted=true&deny_category_prediction=true&from_global=true&text=xiaomi+%D1%81%D0%BC%D0%B0%D1%80%D1%82%D1%84%D0%BE%D0%BD%D1%8B&tf_state=8s_oVf3AfDBplJVDc-KUlaCtFGthxiP1o88FkNI4jY-EaOubT-7pY_u5pMQeP6U3j_c%3D"
main_page_source = get_html(base_url)

soup = BeautifulSoup(main_page_source, 'html.parser')

page_anchors = soup.find('div', class_='e9p').find_all('a', href=True)
pages = list(map(lambda x: urllib.parse.urljoin(base_url, x) if is_relative_url(x) else x, [page_anchor['href'] for page_anchor in page_anchors]))[1:]
print(*pages, sep='\n')


In [101]:
def get_links_from_page(page_html):

    soup = BeautifulSoup(page_html, 'html.parser')
    links = []

    elements = soup.find_all('div', class_="ui3 u3i")
    for element in elements:
        i5u = element.find('div', class_='i5u')
        anchor = i5u.find('a', href=True)
        anchor_url = anchor['href']
        
        if is_relative_url(anchor_url):
            anchor_url = urllib.parse.urljoin(base_url, anchor_url)

        reviews_div = i5u.find_all('span', class_='v1')
        n_reviews = 0

        if reviews_div:
            n_reviews = int(reviews_div[1].text)
        
        if n_reviews > 200:
            links.append((anchor_url, n_reviews))
    return links

In [102]:
links = get_links_from_page(main_page_source)
for page in pages:
    links += get_links_from_page(get_html(page))
    


In [106]:
links

[('https://www.ozon.ru/product/smartfon-xiaomi-redmi-note-12s-8-256-gb-chernyy-965870620/?asb=bS5ZZpen9PkW9GbFrO9Y33QgSHNJGYMa4JBo%252BjToPPw%253D&asb2=agkkgiGM_X6s9Utenw6mh8OhCUYzSAcw9w0qip7FeKoFweDu5YsTxSwULwN86VL1&avtc=1&avte=4&avts=1696870934&keywords=xiaomi+%D1%81%D0%BC%D0%B0%D1%80%D1%82%D1%84%D0%BE%D0%BD%D1%8B',
  1224),
 ('https://www.ozon.ru/product/smartfon-xiaomi-redmi-note-12s-8-256-gb-siniy-965870445/?asb=O1VhT8K6GXkHKvoBEfDB%252BdIboP%252B0okKKNOvGl4Fl1Lw%253D&asb2=_cfvrAUY0E8ZaFzL9HYaCuXyvuCqOqXKhHYr9GZOCNJFhOfXMNepQSLvJOzXHNgi&avtc=1&avte=4&avts=1696870934&keywords=xiaomi+%D1%81%D0%BC%D0%B0%D1%80%D1%82%D1%84%D0%BE%D0%BD%D1%8B',
  1223),
 ('https://www.ozon.ru/product/smartfon-xiaomi-redmi-12c-3-64-gb-seryy-902784453/?asb=wQ0M4ZjnS%252Ba2VjPnI15%252FtHnWLwHCXQoK53nWmovu9n0Lp4BYfZ6L3utxU4Stg5Ho&asb2=IgOvbdpAmMyf7rU7_sEPxLBExCYy3cNCubKHxz0quwktCoISQXo2AZRZav33tfzLDOXDSLU3uppm9OrjoLxRyq2ukaOEwUoadVm74mrrux9xbZq_jCGtEmWndq9fKGfWUGxXhhDtSBnrX_btHmJglTWzworGHbq2gOSOZUGJ4d0&avtc

In [107]:
with open('links.txt', 'w') as f:
    for link, review in links:
        f.write(link + ' ' + str(review) + '\n')

In [149]:
test_html = get_html(links[0][0])

In [112]:
def get_data(page_html):
    soup = BeautifulSoup(page_html, 'html.parser')

    elements = soup.find_all('div', class_="qt")
    for element in elements:
        print(element.text, end='__________')



In [113]:
get_data(test_html)

In [ ]:
print(test_html)